In [1]:
import numpy as np 
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIRHIKAGURUMURTHY\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIRHIKAGURUMURTHY\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('FinalTweetList.csv')
df.columns=['tweet','sentiment','bank']
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

In [3]:
#find all banks - first list out all words beginning with @
def find(text):
    list = re.findall("[@]\w+", text)
    return list
df['@'] = (df['tweet']).apply(lambda x: find(x))
df.head()

,tweet,sentiment,bank,@
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,NaN,[@AxisBankSupport]
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,NaN,[@TheOfficialSBI]
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,NaN,"[@HDFCBank_Cares, @sanatm]"
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,NaN,"[@ICICIBank, @TheOfficialSBI]"
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,NaN,"[@RBI, @TheOfficialSBI]"


In [4]:
#frequency of words beginning with @
#As we need testing data of 500 values we will only take the banks specified in more than 500 rows in the datset - upon analysing we see that 
#these are - SBI,Axis,HDFC,Kotak and ICICI
lis=df['@'].to_list()
flat = [y for x in lis for y in x]
count = Counter(flat)
count.most_common()

[('@TheOfficialSBI', 2468),
 ('@HDFCBank_Cares', 928),
 ('@ICICIBank_Care', 893),
 ('@KotakBankLtd', 691),
 ('@AxisBankSupport', 562),
 ('@RBI', 481),
 ('@ICICIBank', 448),
 ('@HDFC_Bank', 364),
 ('@arunjaitley', 315),
 ('@narendramodi', 312),
 ('@bankofbaroda', 268),
 ('@AxisBank', 262),
 ('@PMOIndia', 248),
 ('@FinMinIndia', 144),
 ('@udaykotak', 82),
 ('@abndnkmtr', 41),
 ('@YESBANK', 36),
 ('@mdpnb', 27),
 ('@SBICard_Connect', 24),
 ('@ndtv', 23),
 ('@IDBI_Bank', 22),
 ('@BJP4India', 20),
 ('@TimesNow', 19),
 ('@DasShaktikanta', 19),
 ('@mediacrooks', 18),
 ('@suchetadalal', 17),
 ('@chandakochhar', 16),
 ('@aajtak', 16),
 ('@htTweets', 15),
 ('@timesofindia', 14),
 ('@ZeeNews', 14),
 ('@HDFCBankNews', 13),
 ('@ArvindKejriwal', 13),
 ('@CKforCleanIndia', 13),
 ('@theofficialsbi', 13),
 ('@neha_aks', 13),
 ('@CNBCTV18Live', 13),
 ('@MyIndusIndBank', 12),
 ('@karishmakotak', 12),
 ('@OfficeOfRG', 12),
 ('@sudhirchaudhary', 11),
 ('@arjunrammeghwal', 11),
 ('@INCIndia', 11),
 ('@ANI_n

In [11]:
#removing punctuation and converting to lower case
def rmvpuct_lower(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    return text_rc

df['Tweet_str'] = df['tweet'].apply(lambda x:rmvpuct_lower(x))
df.head()

,tweet,sentiment,bank,@,Tweet_str,Tweet_cleaned
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,[axis],[@AxisBankSupport],axisbanksupport is permanently busy amp can...,"[axisbanksupport, permanently, busy, amp, cann..."
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,[sbi],[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...,"[theofficialsbi, hours, linestill, waiting, wi..."
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...,"[hdfcbankcares, atm, machines, times, india, n..."
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,"[icici, icici, sbi]","[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...,"[icicibank, thanks, icici, bank, got, case, at..."
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,[sbi],"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...,"[rbi, assam, banks, exchanging, exchanged, yes..."


In [10]:
#finding bank associated with each tweet
def find_bank(text):
    bnk=re.findall(r"hdfc|axis|sbi|icici|kotak",text)
    return bnk
df['bank'] = (df['Tweet_str']).apply(lambda x: find_bank(x))
df.head()

,tweet,sentiment,bank,@,Tweet_str,Tweet_cleaned
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,[axis],[@AxisBankSupport],axisbanksupport is permanently busy amp can...,"[axisbanksupport, permanently, busy, amp, cann..."
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,[sbi],[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...,"[theofficialsbi, hours, linestill, waiting, wi..."
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...,"[hdfcbankcares, atm, machines, times, india, n..."
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,"[icici, icici, sbi]","[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...,"[icicibank, thanks, icici, bank, got, case, at..."
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,[sbi],"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...,"[rbi, assam, banks, exchanging, exchanged, yes..."


In [12]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
df['Tweet_cleaned'] = df['tweet'].apply(lambda x: clean_text(x))

In [13]:
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_cleaned'].apply(lambda x: stemming(x))

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df['Tweet_lemmatized_stemmed'] = df['Tweet_stemmed'].apply(lambda x: lemmatizer(x))
df.head()



,tweet,sentiment,bank,@,Tweet_str,Tweet_cleaned,Tweet_stemmed,Tweet_lemmatized_stemmed
0,@AxisBankSupport 18..233.... is permanently bu...,displeasure,[axis],[@AxisBankSupport],axisbanksupport is permanently busy amp can...,"[axisbanksupport, permanently, busy, amp, cann...","[axisbanksupport, perman, busi, amp, cannot, g...","[axisbanksupport, perman, busi, amp, cannot, g..."
1,"@TheOfficialSBI 3 hours in line,still waiting....",displeasure,[sbi],[@TheOfficialSBI],theofficialsbi hours in linestill waiting wis...,"[theofficialsbi, hours, linestill, waiting, wi...","[theofficialsbi, hour, linestil, wait, wish, u...","[theofficialsbi, hour, linestil, wait, wish, u..."
2,@HDFCBank_Cares Both the ATM machines here in ...,displeasure,[hdfc],"[@HDFCBank_Cares, @sanatm]",hdfcbankcares both the atm machines here in ti...,"[hdfcbankcares, atm, machines, times, india, n...","[hdfcbankcar, atm, machin, time, india, noida,...","[hdfcbankcar, atm, machin, time, india, noida,..."
3,@ICICIBank Thanks icici bank I got case in you...,miscellaneous,"[icici, icici, sbi]","[@ICICIBank, @TheOfficialSBI]",icicibank thanks icici bank i got case in your...,"[icicibank, thanks, icici, bank, got, case, at...","[icicibank, thank, icici, bank, got, case, atm...","[icicibank, thank, icici, bank, got, case, atm..."
4,@RBI In Assam all banks are exchanging 4000/- ...,displeasure,[sbi],"[@RBI, @TheOfficialSBI]",rbi in assam all banks are exchanging once on...,"[rbi, assam, banks, exchanging, exchanged, yes...","[rbi, assam, bank, exchang, exchang, yesterday...","[rbi, assam, bank, exchang, exchang, yesterday..."


In [138]:
data=df[['tweet','Tweet_str','Tweet_stemmed','sentiment','bank']]


In [139]:
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder() 
data['sentiment']= le.fit_transform(data['sentiment'])

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(data[['sentiment']]).toarray())
# merge with main df bridge_df on key values
data = data.join(enc_df)
data

C:\Users\KIRHIKAGURUMURTHY\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,tweet,Tweet_str,Tweet_stemmed,sentiment,bank,0,1,2
0,@AxisBankSupport 18..233.... is permanently bu...,axisbanksupport is permanently busy amp can...,"[axisbanksupport, perman, busi, amp, cannot, g...",1,[axis],0.0,1.0,0.0
1,"@TheOfficialSBI 3 hours in line,still waiting....",theofficialsbi hours in linestill waiting wis...,"[theofficialsbi, hour, linestil, wait, wish, u...",1,[sbi],0.0,1.0,0.0
2,@HDFCBank_Cares Both the ATM machines here in ...,hdfcbankcares both the atm machines here in ti...,"[hdfcbankcar, atm, machin, time, india, noida,...",1,[hdfc],0.0,1.0,0.0
3,@ICICIBank Thanks icici bank I got case in you...,icicibank thanks icici bank i got case in your...,"[icicibank, thank, icici, bank, got, case, atm...",2,"[icici, icici, sbi]",0.0,0.0,1.0
4,@RBI In Assam all banks are exchanging 4000/- ...,rbi in assam all banks are exchanging once on...,"[rbi, assam, bank, exchang, exchang, yesterday...",1,[sbi],0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
5688,@HDFCBank_Cares over 25 calls this week.I gues...,hdfcbankcares over calls this weeki guess ull...,"[hdfcbankcar, call, weeki, guess, ull, dont, t...",1,[hdfc],0.0,1.0,0.0
5689,@ICICIBank_Care e-Pay slip as system not worki...,icicibankcare epay slip as system not working ...,"[icicibankcar, epay, slip, system, work, sec, ...",1,[icici],0.0,1.0,0.0
5690,@TheOfficialSBI staff at Sbi Akaltara branch C...,theofficialsbi staff at sbi akaltara branch ch...,"[theofficialsbi, staff, sbi, akaltara, branch,...",2,"[sbi, sbi]",0.0,0.0,1.0
5691,"@singhraviranja4 @AxisBankSupport Hi, sorry fo...",singhraviranja axisbanksupport hi sorry for th...,"[singhraviranja, axisbanksupport, hi, sorri, t...",2,[axis],0.0,0.0,1.0


In [140]:
hdfc=data[data.apply(lambda x: 'hdfc' in x['bank'],axis=1)]
axis=data[data.apply(lambda x: 'axis' in x['bank'],axis=1)]
sbi=data[data.apply(lambda x: 'sbi' in x['bank'],axis=1)]
kotak=data[data.apply(lambda x: 'kotak' in x['bank'],axis=1)]
icici=data[data.apply(lambda x: 'icici' in x['bank'],axis=1)]

In [141]:
hdfc_test =hdfc.iloc[0:500]
hdfc_train=hdfc.iloc[500:]
axis_test =axis.iloc[0:500]
axis_train=axis.iloc[500:]
sbi_test =sbi.iloc[0:500]
sbi_train=sbi.iloc[500:]
kotak_test =kotak.iloc[0:500]
kotak_train=kotak.iloc[500:]
icici_test =icici.iloc[0:500]
icici_train=icici.iloc[500:]

In [142]:
train_data = pd.concat([hdfc_train,axis_train,sbi_train,kotak_train,icici_train])
test_data=pd.concat([hdfc_test,axis_test,sbi_test,kotak_test,icici_test])

In [143]:
from sklearn.model_selection import train_test_split
tr=train_data[['Tweet_str','Tweet_stemmed','sentiment',0,1,2]]
tr.rename(columns = {'Tweet_stemmed':'tokens'}, inplace = True) 


C:\Users\KIRHIKAGURUMURTHY\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [144]:
all_training_words = [word for tokens in tr["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

39529 words total, with a vocabulary size of 6104
Max sentence length is 22


3884words total, with a vocabulary size of 1404
Max sentence length is 21


In [182]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
word2vec_path = 'word2vec_100.model'
word2vec = KeyedVectors.load(word2vec_path)

In [146]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(tr["Tweet_str"].tolist())
training_sequences = tokenizer.texts_to_sequences(tr["Tweet_str"].tolist())
train_word_index = tokenizer.word_index
print("Found %s unique tokens." % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, maxlen=50)
train_cnn_data

Found 7322 unique tokens.


array([[   0,    0,    0, ..., 2050, 2051, 3208],
       [   0,    0,    0, ...,    6,   25,  179],
       [   0,    0,    0, ...,  531,  167,   84],
       ...,
       [   0,    0,    0, ...,   62,   64,   62],
       [   0,    0,    0, ...,    6,  523,   11],
       [   0,    0,    0, ...,  376,    4,  297]])

In [147]:
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 100))
for word,index in train_word_index.items():
 train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(100)
print(train_embedding_weights.shape)




(7323, 100)


C:\Users\KIRHIKAGURUMURTHY\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
C:\Users\KIRHIKAGURUMURTHY\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [61]:
embedding_vector_length = 100 
model = Sequential() 
model.add(Embedding(7323, embedding_vector_length, input_length=50)) 
model.add(LSTM(100)) 
model.add(Dense(3, activation='softmax')) 
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 50, 100)           732300    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 813,003
Trainable params: 813,003
Non-trainable params: 0
_________________________________________________________________
None


In [55]:
"""""model=Sequential()
embedding_layer=Embedding(7323,100,weights=[train_embedding_weights],input_length=50,trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])"""

In [148]:
import numpy as np
VALIDATION_SPLIT=0.2

from keras.utils import to_categorical

indices=np.arange(train_cnn_data.shape[0])
np.random.shuffle(indices)
tweet_pad=train_cnn_data[indices]
sentiment=tr['sentiment'].values
sentiment=sentiment[indices]
num_validation_samples=int(VALIDATION_SPLIT * tweet_pad.shape[0])
X_train_pad=tweet_pad[:-num_validation_samples]
y_train=sentiment[:-num_validation_samples]
#y_train = to_categorical(y_train)
X_test_pad=tweet_pad[-num_validation_samples:]
y_test=sentiment[-num_validation_samples:]
#y_test= to_categorical(y_test)


Train on 2904 samples, validate on 323 samples
Epoch 1/4
2904/2904 [==============================] - 10s 3ms/step - loss: 0.1286 - acc: 0.9556 - val_loss: 0.6128 - val_acc: 0.7709
Epoch 2/4
2904/2904 [==============================] - 10s 4ms/step - loss: 0.0927 - acc: 0.9720 - val_loss: 0.6932 - val_acc: 0.7564
Epoch 3/4
2904/2904 [==============================] - 10s 4ms/step - loss: 0.1194 - acc: 0.9540 - val_loss: 0.6925 - val_acc: 0.7585
Epoch 4/4
2904/2904 [==============================] - 11s 4ms/step - loss: 0.1211 - acc: 0.9555 - val_loss: 0.6551 - val_acc: 0.7637


In [149]:
model.fit(X_train_pad,y_train,batch_size=128,epochs=25,validation_data=(X_test_pad,y_test),verbose=2)

Train on 2869 samples, validate on 717 samples
Epoch 1/25
 - 3s - loss: 0.2710 - accuracy: 0.9292 - val_loss: 0.2433 - val_accuracy: 0.9219
Epoch 2/25
 - 3s - loss: 0.1076 - accuracy: 0.9690 - val_loss: 0.2502 - val_accuracy: 0.9247
Epoch 3/25
 - 3s - loss: 0.0486 - accuracy: 0.9881 - val_loss: 0.2598 - val_accuracy: 0.9303
Epoch 4/25
 - 3s - loss: 0.0287 - accuracy: 0.9930 - val_loss: 0.2934 - val_accuracy: 0.9289
Epoch 5/25
 - 3s - loss: 0.0176 - accuracy: 0.9962 - val_loss: 0.3013 - val_accuracy: 0.9289
Epoch 6/25
 - 3s - loss: 0.0114 - accuracy: 0.9965 - val_loss: 0.3148 - val_accuracy: 0.9233
Epoch 7/25
 - 3s - loss: 0.0081 - accuracy: 0.9979 - val_loss: 0.3432 - val_accuracy: 0.9149
Epoch 8/25
 - 3s - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.3484 - val_accuracy: 0.9149
Epoch 9/25
 - 3s - loss: 0.0064 - accuracy: 0.9979 - val_loss: 0.3665 - val_accuracy: 0.9163
Epoch 10/25
 - 3s - loss: 0.0055 - accuracy: 0.9976 - val_loss: 0.3721 - val_accuracy: 0.9121
Epoch 11/25
 - 3s - lo

In [157]:
#hdfc_score=hdfc_test['Tweet_str']
all_training_words_bank = [word for tokens in test_data["Tweet_stemmed"] for word in tokens]
training_sentence_lengths_bank = [len(tokens) for tokens in test_data["Tweet_stemmed"]]
TRAINING_VOCAB_bank = sorted(list(set(all_training_words_bank)))
tokenizer_bank = Tokenizer(num_words=len(TRAINING_VOCAB_bank), lower=True, char_level=False)
tokenizer_bank.fit_on_texts(test_data["Tweet_str"].tolist())
training_sequences_bank = tokenizer_bank.texts_to_sequences(test_data["Tweet_str"].tolist())
train_word_index_bank = tokenizer_bank.word_index
train_cnn_data_bank = pad_sequences(training_sequences_bank, maxlen=50)
train_cnn_data_bank
pred=model.predict(train_cnn_data_bank)
pred.tolist()

[[0.010108078829944134, 0.026802001520991325, 0.9630899429321289],
 [0.00022033959976397455, 0.9990819692611694, 0.0006977419252507389],
 [3.017194649146404e-05, 0.9829550385475159, 0.017014913260936737],
 [4.075426204508403e-06, 0.9998537302017212, 0.00014225050108507276],
 [4.909994004265172e-06, 0.9999004602432251, 9.462966409046203e-05],
 [4.625561632565223e-05, 0.019995415583252907, 0.9799583554267883],
 [0.002144693396985531, 9.246066474588588e-05, 0.9977627992630005],
 [0.00010781000310089439, 0.8932702541351318, 0.10662198811769485],
 [0.01789962500333786, 0.04068601131439209, 0.9414143562316895],
 [1.1607819942582864e-05, 0.00019443212659098208, 0.9997939467430115],
 [0.024245191365480423, 0.5428218841552734, 0.43293294310569763],
 [1.2297382454562467e-05, 0.9999290704727173, 5.867323852726258e-05],
 [0.009651140309870243, 0.021593226119875908, 0.9687556028366089],
 [0.00017047532310243696, 0.9131069183349609, 0.08672261238098145],
 [0.0007129417499527335, 5.438648076960817e-0

In [158]:
#predictions
maxindex=np.argmax(pred,axis=1)
maxindex
pred_sent=maxindex.tolist()
bank_pred=test_data[['Tweet_str']]
bank_pred=bank_pred.reset_index()
del bank_pred['index']
#f_pred=pd.DataFrame(df_pred)
#dfc_pred['sentiment_pred']=df_pred
#dfc_pred""""""
bank_pred['sentiment_score']=pred_sent
bank_pred
#fc_pred['sentiment_pred']=df_pred
#dfc_pred
#dfc_pred.loc[hdfc_pred.sentiment_pred==1,'sentiment_pred']='displeasure'
#dfc_pred.loc[hdfc_pred.sentiment_pred==0,'sentiment_pred']='miscellaneous'
#dfc_pred.loc[hdfc_pred.sentiment_pred==2,'sentiment_pred']='compliment'
#dfc_pred

,Tweet_str,sentiment_score
0,hdfcbankcares both the atm machines here in ti...,2
1,hdfcbank hdfcbankcares received an email sayin...,1
2,hdfcbankcares the atm at sector charkop kandiv...,1
3,icicibankcare hdfcbankcares axisbank nomoney i...,1
4,rt tarun thanks to hdfcbank hdfcbankcares mgmt...,1
...,...,...
2495,icicibankcare thanks atms were working today,2
2496,icicibankcare your patna zonal office car loan...,1
2497,unfortunately in queue do not need cash hdfcb...,2
2498,icicibankcare chandakochar icicibank dahisar w...,2


In [167]:
hdfc_sent_score_df=bank_pred.iloc[0:500,1]
hdfc_sent_score=hdfc_sent_score_df.sum()
print("HDFC Performance index score is :",hdfc_sent_score)

HDFC Performance index score is : 640


In [168]:
axis_sent_score_df=bank_pred.iloc[500:1000,1]
axis_sent_score=axis_sent_score_df.sum()
print("AXIS Performance index score is :",axis_sent_score)

AXIS Performance index score is : 630


In [170]:
sbi_sent_score_df=bank_pred.iloc[1000:1500,1]
sbi_sent_score=sbi_sent_score_df.sum()
print("SBI Performance index score is :",sbi_sent_score)

SBI Performance index score is : 676


In [171]:
kotak_sent_score_df=bank_pred.iloc[1500:2000,1]
kotak_sent_score=kotak_sent_score_df.sum()
print("HDFC Performance index score is :",kotak_sent_score)

HDFC Performance index score is : 659


In [172]:
icici_sent_score_df=bank_pred.iloc[2000:2500,1]
icici_sent_score=icici_sent_score_df.sum()
print("ICICI Performance index score is :",icici_sent_score)

ICICI Performance index score is : 685


In [174]:
import pickle
save_classifier = open("DL1.pickle","wb")
pickle.dump(model, save_classifier)
save_classifier.close() #saving the classifier

In [175]:
classifier_f = open("DL1.pickle", "rb") #loading the saved classifier
classifier = pickle.load(classifier_f)
classifier_f.close() 

2500/2500 [==============================] - 1s 315us/step


[3.757430519104004, 0.3968000113964081]

IndexError: index 2991 is out of bounds for axis 0 with size 2500